# Record Linkage
---------


### Author Information
**Author:** PJ Gibson  
**Email:** Peter.Gibson@doh.wa.gov  
**Github:**   https://github.com/DOH-PJG1303

### Project Information
**Created Date:** 2023-05-26  
**Last Updated:** 2023-05-26  
**Version:** 1  

### Description
This notebook should serve to educate newcomers to Python on Machine Learning in the context of Record Linkage.

### Notes


## 1. Import Libraries

In [47]:
# Standard data analysis tools
import pandas as pd
import numpy as np

# Record linkage specific resources
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonetic
from recordlinkage.index import Block

# Machine learning tools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


## 2. Read Data

Here, we're using synthetic data that I created to replicate Oregon's population.
The data represents the population of individuals in the years 2020-2022 who were born in Coos County in Oregon.
They may live elsewhere.
Interesting features that this training data includes:
* several people can live in the same building
* families exist.  If a child is <18, they live with their parent or parents
* fields change dependent on the year. If someone gets married and changes their name in 2020, their data in 2019 will look different than their data in 2020 for the lname field.

For more information, reach out to PJ and he'd be happy to elaborate.
Again, this data is synthetic.

In [28]:
df1 = pd.read_csv('Data/synthetic_df1.csv', dtype=str)
df2 = pd.read_csv('Data/synthetic_df2.csv', dtype=str)

## 3. Record Linkage Steps

In [45]:
# Clean the data
for col in ['fname', 'lname', 'dob', 'phone', 'add']:
    df1[col] = clean(df1[col])
    df2[col] = clean(df2[col])

# Generate metaphone versions of the fields
for col in ['fname', 'lname']:
    df1['meta_'+col] = phonetic(df1[col], method='metaphone')
    df2['meta_'+col] = phonetic(df2[col], method='metaphone')

# Create the index (pairs of records to compare)
indexer = rl.Index()

# Generate a blocking scheme as a union of the following blocks
indexer.add(Block('dob'))
indexer.add(Block(['meta_fname', 'meta_lname']))
indexer.add(Block('building_id'))
indexer.add(Block('parents_partnership_id'))

pairs = indexer.index(df1, df2)

# Create the Compare object
compare_precursor = rl.Compare()
compare = rl.Compare()

##############################################################################################################

# Calculate the average similarity score for each field and use it as the missing value
for col in ['fname', 'lname', 'dob', 'add']:
    compare_precursor.string(col, col, method='jarowinkler', missing_value=-1, label=col)

# Compute the comparison scores
features_precursor = compare_precursor.compute(pairs, df1, df2)

##############################################################################################################

# Calculate the average similarity score for each field
for col in ['fname', 'lname', 'dob', 'add']:
    average_score = features_precursor.replace(-1,None)[col].mean()
    compare.string(col, col, method='jarowinkler', missing_value=average_score, label=col)

# Compare the phone fields using damerau_levenshtein similarity
compare.string('phone', 'phone', method='damerau_levenshtein', label='phone')
compare.exact('ssn','ssn',label='label')

# Compute the comparison scores
features = compare.compute(pairs, df1, df2)


## 4. Final Preprocessing Step

In [55]:
# Separate majority and minority classes
df_majority = features[features.label==0]
df_minority = features[features.label==1]

In [56]:
# Discretize the fields into bins
for col in ['fname', 'lname', 'dob', 'phone', 'add']:
    df_majority[col + '_bin'] = pd.qcut(df_majority[col], q=3, duplicates='drop')

# Create a 'strata' column that combines the bins
df_majority['strata'] = df_majority[['fname_bin', 'lname_bin', 'dob_bin', 'phone_bin', 'add_bin']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

# Sample from each stratum
samples = []
for stratum, group in df_majority.groupby('strata'):
    samples.append(group.sample(min(len(group), 100000 // df_majority['strata'].nunique()), random_state=42))

# Concatenate the samples into a single dataframe
df_majority_sampled = pd.concat(samples)

# Drop the bin and strata columns
df_majority_sampled = df_majority_sampled.drop(['fname_bin', 'lname_bin', 'dob_bin', 'phone_bin', 'add_bin', 'strata'], axis=1)

<ipython-input-56-74820ad1eaa5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_majority[col + '_bin'] = pd.qcut(df_majority[col], q=3, duplicates='drop')
<ipython-input-56-74820ad1eaa5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_majority[col + '_bin'] = pd.qcut(df_majority[col], q=3, duplicates='drop')
<ipython-input-56-74820ad1eaa5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [ ]:
# Upsample minority class
df_minority_sampled = df_minority.sample(100000, replace=True, random_state=42)

In [ ]:
# Combine majority class with upsampled minority class
df_resampled = pd.concat([df_majority_downsampled, df_minority_upsampled])

In [ ]:
df_resampled.to_csv('./Data/synthetic_training_data.csv',header=True,index=True))

## 4. Machine Learning

In [46]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


fname     lname       dob       add  phone  label
0      0      0.925000  1.000000  1.000000  0.987500    0.6      1
       226    0.000000  0.455556  1.000000  0.504762    0.0      0
       62543  0.000000  1.000000  0.840000  1.000000    0.6      0
       62544  0.000000  1.000000  0.840000  0.980000    0.6      0
       98733  0.650794  1.000000  0.786667  1.000000    0.6      0
...                ...       ...       ...       ...    ...    ...
105969 86088  0.527778  0.000000  0.622222  0.485079    0.2      0
       93281  0.000000  0.522222  0.533333  0.531746    0.1      0
       93282  0.000000  0.522222  0.533333  0.571429    0.1      0
       99479  0.000000  0.000000  0.752381  0.468651    0.1      0
       99480  0.000000  0.000000  0.752381  0.468651    0.1      0

[4264761 rows x 6 columns]

In [48]:
features.label.value_counts()

0    4053909
1     210852
Name: label, dtype: int64

In [22]:
df1_cleaned = clean_datasets(df1).add_suffix('_1')
df2_cleaned = clean_datasets(df2).add_suffix('_2')

In [19]:
blockingSchema = []


block_dob = pd.merge(left=df1_cleaned, right=df2_cleaned, left_on='dob_1')

,ssn,fname,lname,dob,phone,add,unique_id,parents_partnership_id,house_id,building_id
0,001-09-7165,asmy,stowe,2021-04-13,5415064522,232 laksewood dr,111669149696,1274418,163208848114,2389704
1,001-96-0107,yvette,smith,2013-08-19,5413639674,63461 s jade rd,111669149697,1158240,146028991471,3328362
2,001-96-0107,yvette,smith,2013-08-19,5413639674,63461 s jade rd,111669149698,1158240,146028991471,3328362
3,002-87-4113,eve,ayres,2016-07-11,5416441986,3097 pacific loop,111669149699,1245069,154618902919,2810391
4,002-87-4113,evellyn,ayres,2016-07-11,5416441986,3097 pacific loop,111669149700,1245069,154618902919,2810391
...,...,...,...,...,...,...,...,...,...,...
105965,996-71-8333,wesley,park,2020-06-12,NaN,94271 lord ln,7065,1240220,154618902874,192312
105966,996-92-1738,lucas,connelly,2019-04-16,5415893007,none,7066,1261831,188978681781,2025198
105967,996-92-1738,lucas,connelly,2019-04-16,5415893007,182 airport way,7067,1261831,188978681781,2025198
105968,998-03-1555,angelina,fiscus,2014-09-23,5413110620,976 w lockhart ave,7068,1025273,171798818513,1276349
